In [ ]:
import sys 
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Clearvale/jake_scripts/Amy_flywheel_scripts/')

import numpy as np
import pandas as pd
import os
import seaborn as sns

In [ ]:
all_metrics_z = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/rest/rest_all_metrics_z.csv')

In [ ]:
import pandas as pd
from sklearn.preprocessing import power_transform

# Assuming sub_data is your DataFrame and filtered_data is already defined
# filtered_data = sub_data.filter(regex='disp')
# Filter columns based on regex
filtered_data = all_metrics_z.iloc[:, 1:]

# Apply Yeo-Johnson transformation
# Note: power_transform returns a numpy array, so we need to convert it back to a DataFrame
yeo_johnson_transformed_data = pd.DataFrame(power_transform(filtered_data, method='yeo-johnson'), columns=filtered_data.columns)

#yeo_johnson_transformed_data

import numpy as np
from scipy import stats

# Perform the Shapiro-Wilk test for normality
shapiro_test = stats.shapiro(yeo_johnson_transformed_data)

print(f"Shapiro-Wilk Test statistic: {shapiro_test.statistic}, p-value: {shapiro_test.pvalue}")

# Interpretation of the result
if shapiro_test.pvalue > 0.05:
    print("The data is likely normal (fail to reject H0)")
else:
    print("The data is likely not normal (reject H0)")

In [ ]:
all_metrics_z_yeo = pd.concat([all_metrics_z[['SubID']], yeo_johnson_transformed_data], axis=1)

In [ ]:
def run_regression(data, target, y_vars, interaction=None):
    import pandas as pd
    import statsmodels.api as sm
    import statsmodels.formula.api as smf

    if interaction is not None:
        # Creating the formula with interaction terms if there are multiple y_vars
        if len(y_vars) > 1:
            joined_vars = ' * '.join(y_vars)
        else:
            joined_vars = y_vars[0]
        
        new_y_vars = [joined_vars]
        formula = f'{target[0]} ~ {joined_vars}'

    else:
        joined_vars = ' + '.join(y_vars)
        new_y_vars = [joined_vars]
        formula = f'{target[0]} ~ {joined_vars}'

    # Fit the regression model using the formula
    model = smf.ols(formula=formula, data=data).fit()

    # Print the full regression output
    summary = model.summary()

    var = pd.DataFrame(summary.tables[0].data).iloc[0, 1]

    table1 = pd.DataFrame(summary.tables[0].data).iloc[:4, 3:].T.assign(var=var)
    table1.columns = ['r2', 'adjr2', 'fstat', 'pval', 'var']
    table1 = table1[['var', 'r2', 'adjr2', 'fstat', 'pval']]

    table2_cols = pd.DataFrame(summary.tables[1].data).loc[0].to_list() + ['var']
    table2_cols[0] = 'parameter'
    table2 = pd.DataFrame(summary.tables[1].data).iloc[1:].assign(var=var)
    table2.columns = table2_cols
    table2 = table2[['var', 'parameter', 'coef', 'std err', 't', 'P>|t|']]

    df = pd.merge(table1, table2, how='outer', left_on='var', right_on='parameter')

    # Combine 'var_x' and 'var_y' into a new column 'var'
    df['var'] = df['var_x'].fillna(df['var_y'])

    # Drop the original 'var_x' and 'var_y' columns
    df.drop(columns=['var_x', 'var_y'], inplace=True)

    # Reorder columns to place 'var' at the front
    cols = ['var'] + [col for col in df.columns if col != 'var']
    df = df[cols]
    df = df.assign(formula=formula)

    return df


def regression_function(data, variable_list, interaction):

    import itertools

    # Given list
    variables = variable_list
    targets = [['brd_z_ave']]

    # Create a list to hold all combinations
    all_combinations = []

    # Generate combinations for each length from 1 to the length of the list
    for r in range(1, len(variables) + 1):
        combinations = list(itertools.combinations(variables, r))
        all_combinations.extend(combinations)

    # Convert each tuple to a list
    list_combinations = [list(item) for item in all_combinations]

    regression_combinations = []
    for i in targets:
        for j in list_combinations:
            regression_combinations.append(run_regression(data, i, j, interaction))

    output_regressions = pd.concat(regression_combinations)

    # List of columns to convert
    columns_to_convert = ['r2', 'adjr2', 'fstat', 'pval', 'coef', 'std err', 't', 'P>|t|']

    # Convert each specified column to numeric, handling non-numeric values by converting them to NaN
    for column in columns_to_convert:
        output_regressions[column] = pd.to_numeric(output_regressions[column], errors='coerce')

    return output_regressions

from concurrent.futures import ProcessPoolExecutor, as_completed

def process_sample(data, columns):
    # Sample with replacement
    sample = data.sample(n=len(data), replace=True)
    # Run the regression function
    result = regression_function(sample, columns, interaction=None)
    return result

def bootstrap_regression_parallel(data, columns, n_iterations=1000):
    results = []
    with ProcessPoolExecutor() as executor:
        # Submit all tasks to the executor
        futures = [executor.submit(process_sample, data, columns) for _ in range(n_iterations)]
        
        # As each future completes, gather results
        for future in as_completed(futures):
            results.append(future.result())
    
    # Concatenate all results into a single DataFrame
    final_results = pd.concat(results, ignore_index=True)
    
    return final_results

In [ ]:
all_list = ['vn_smn_disp', 'fpcn_dmn_disp', 'smn_fpcn_disp',
       'vn_fpcn_disp', 'smn_dmn_disp', 'vn_dmn_disp', 'vn_disp', 'smn_disp',
       'fpcn_disp', 'dmn_disp', 'vn_ecc', 'smn_ecc', 'fpcn_ecc', 'dmn_ecc',
       'vn_eigcent', 'smn_eigcent', 'fpcn_eigcent', 'dmn_eigcent', 'vn_mod',
       'smn_mod', 'fpcn_mod', 'dmn_mod', 'rest_within_disp',
       'rest_across_disp', 'rest_ecc', 'rest_eigcent', 'rest_mod']

In [ ]:
import pandas as pd
import numpy as np

def bootstrap_regression(data, columns, n_iterations=1000):
    # Store results in a list of DataFrames
    
    global regression_function
    
    results = []
    # Perform bootstrapping
    for i in range(n_iterations):
        
        # Sample with replacement
        sample = data.sample(n=len(data), replace=True)

        # Run the regression function
        result = regression_function(sample, columns, interaction=None)
        print
        # Append the result to the list
        results.append(result)
    
    # Concatenate all results into a single DataFrame
    final_results = pd.concat(results, ignore_index=True)
    
    return final_results


test_list = ['vn_smn_disp', 'fpcn_dmn_disp', 'smn_fpcn_disp',
       'vn_fpcn_disp', 'smn_dmn_disp', 'vn_dmn_disp', 'vn_disp', 'smn_disp',
       'fpcn_disp', 'dmn_disp', 'vn_ecc', 'smn_ecc', 'fpcn_ecc', 'dmn_ecc',
       'rest_within_disp','rest_across_disp', 'rest_ecc', 'rest_eigcent']

# Call the bootstrap function with your dataset, regression function, and the list of all features
bootstrapped_results = bootstrap_regression(all_metrics_z, test_list, n_iterations=100)

In [ ]:
bootstrapped_results.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/rest/rest_regressions/bootstrap_regressions/bootstrap_regressions/boot_results.csv')



In [ ]:
bootstrapped_results = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/rest/rest_regressions/bootstrap_regressions/bootstrap_regressions/boot_results.csv')



In [ ]:
import pandas as pd
import seaborn as sns
# Assuming bootstrapped_results is already loaded as a DataFrame
# First, let's group by 'formula' and calculate the mean and standard deviation of 'pval'
# Assuming 'fstat' and 'adjr2' are columns in bootstrapped_results DataFrame
# Group by 'formula' and calculate the mean and standard deviation of 'pval', and mean of 'fstat' and 'adjr2'
stats = bootstrapped_results.groupby('formula').agg({
    'pval': ['mean', 'std'],
    'fstat': ['mean', 'std'],
    'adjr2': ['mean', 'std']
})

# Rename columns for clarity
stats.columns = ['mean_pval', 'std_pval', 'mean_fstat', 'std_fstat', 'mean_adjr2', 'std_adjr2']

# Sort the formulas first by mean_pval in ascending order (lower means better),
# and then by std_pval in ascending order (lower variability is better)
sorted_stats = stats.sort_values(by=['mean_pval', 'std_pval'])

# Now, select the top 10 formulas
top_10_formulas = sorted_stats.head(10).reset_index()

top_10_formulas.formula = top_10_formulas.formula.str.replace('brd_z_ave ~', '')

In [ ]:
filter_formulas = top_10_formulas.formula.to_list()
filter_formulas = ['brd_z_ave ~' + item for item in filter_formulas]

top_10_boot_adjr2 = bootstrapped_results.query('formula in @filter_formulas')[['adjr2', 'formula']].dropna()
top_10_boot_pval = bootstrapped_results.query('formula in @filter_formulas')[['pval', 'formula']].dropna()
top_10_boot_fstat = bootstrapped_results.query('formula in @filter_formulas')[['fstat', 'formula']].dropna()

In [ ]:
top_10_boot_stats = pd.concat([top_10_boot_adjr2, top_10_boot_fstat, top_10_boot_pval], axis=1).iloc[:, [1,0,2,4]]

In [ ]:
top_5_formulas = ['brd_z_ave ~' + item for item in top_10_formulas.head(5).formula.to_list()]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Melting the DataFrame to make it suitable for seaborn's factorplot
df_melted = top_10_boot_stats.melt(id_vars=['formula'], value_vars=['adjr2', 'pval', 'fstat'],
                                   var_name='Metric', value_name='Value')

stat_names = {
'pval': 'p',
'fstat': 'F',
'adjr2': 'adj. r2'
}

df_melted.Metric = df_melted.Metric.map(stat_names)

df_melted = df_melted.query('formula in @top_5_formulas')

df_melted.formula = df_melted.formula.str.replace('brd_z_ave ~', '')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Creating the facet grid plot without a legend
g = sns.catplot(x='Value', y='formula', hue='Metric', data=df_melted, kind='point', 
                height=5, aspect=1, join=False, palette='viridis', col='Metric', 
                sharex=False, legend=False, col_wrap=1)

# Enhancing the plot
g.set_axis_labels("Value", "Formula")
# Adjust tick rotation if necessary
#plt.xticks(rotation=45)

# Removing "Metric = " from the plot titles
for ax in g.axes.flat:
    if ax.get_title():
        ax.set_title(ax.get_title().split(' = ')[-1])

# Set up the figure's layout
plt.tight_layout()

# Define save path
save_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/rest/rest_regressions/bootstrap_regressions/figures/top_5_formulas.png'

# Use the FacetGrid's savefig method
g.savefig(save_path, dpi=300, bbox_inches='tight')  # bbox_inches='tight' can help with cutting issues

# Display the plot
plt.show()


In [ ]:
#top_10_formulas.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/operation_regressions/bootstrap_regressions/top_10_formulas.csv')

In [ ]:
reg_vars = ['dmn_mean', 'vn_mean_ecc', 'dmn_mean_ecc', 'global_acc', 'fpcn_dmn_mean']

run_regression(sub_disp_ecc_acc_ev_means, ['brd_z_ave'], reg_vars, interaction=None)

In [ ]:
import statsmodels.formula.api as smf

model = smf.ols(formula='brd_z_ave ~ dmn_mean + vn_mean_ecc + dmn_mean_ecc + global_acc + fpcn_dmn_mean', 
                data=sub_disp_ecc_acc_ev_means).fit()

model.summary()

In [ ]:
sub_disp_ecc_acc_ev_means[['brd_z_ave', 'dmn_mean', 'vn_mean_ecc', 'dmn_mean_ecc', 'global_acc', 'fpcn_dmn_mean']].corr()

In [ ]:
sub_disp_ecc_acc_ev_means[['brd_z_ave', 'dmn_mean', 'fpcn_mean', 'vn_mean', 'smn_mean']].corr()

In [ ]:
def reg_plots(data, x_vars, yvar, color, title, formula_title, save=None):

    import pandas as pd
    #import statsmodels.api as sm
    import statsmodels.formula.api as smf
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Set the entire script to use Arial
    plt.rcParams['font.family'] = 'sans-serif'
    
    joined_vars = ' + '.join(x_vars)
    new_y_vars = [joined_vars]
    formula = f'{yvar[0]} ~ {joined_vars}'

    # Fit the regression model using the formula
    model = smf.ols(formula=formula, data=data).fit()

    # Prepare the design matrix X with multiple x_vars
    #X = sm.add_constant(data[x_vars])  # Adding a constant
    #model = sm.OLS(data[yvar], X).fit()
    data['Predicted'] = model.predict()  # Store the predicted values

    # Adjust figure size
    fig, ax = plt.subplots(figsize=(8, 6))  # Increased figure size

    # Scatter plot with regression line, ensuring CI is shown
    sns.set_theme(style="white")

    # Scatter plot with regression line modifications
    sns.regplot(y='Predicted', x=f'{yvar[0]}', data=data, ci=95,
                color=color, scatter_kws={'s': 40, 'alpha': 0.6}, truncate=False, ax=ax,
                line_kws={'linewidth': 2.5, 'alpha': 0.4})  # Set line width and transparency

    sns.regplot(y='Predicted', x=f'{yvar[0]}', data=data, fit_reg=False,
                color='white', scatter_kws={'s': 20, 'alpha': .5}, truncate=False, ax=ax)


   # Set axis labels
    ax.set_xlabel('Thought Problems', fontsize=15)
    ax.set_ylabel('Predicted', fontsize=15)

    # Set title using text for precise control
    ax.text(.02, 1.07, title, transform=ax.transAxes, fontsize=15, fontweight='bold', verticalalignment='top')

    # Adding subtitle with F-statistic and p-value using text annotation, with 'p' in italic
    subtitle = f"F={model.fvalue:.3f}, "
    ax.text(.02, 1, subtitle, transform=ax.transAxes, fontsize=14, verticalalignment='top')
    ax.text(.02 + ax.transAxes.inverted().transform(ax.transData.transform((-1.13, 0)))[0], 1, "p", transform=ax.transAxes, fontsize=14, verticalalignment='top', style='italic')
    ax.text(.02 + ax.transAxes.inverted().transform(ax.transData.transform((-1.06, 0)))[0], 1, f"={model.f_pvalue:.3f}", transform=ax.transAxes, fontsize=14, verticalalignment='top')

    ax.text(.02 + ax.transAxes.inverted().transform(ax.transData.transform((-.7, 0)))[0], 1, ", adj. r2", transform=ax.transAxes, fontsize=14, verticalalignment='top', style='italic')
    ax.text(.02 + ax.transAxes.inverted().transform(ax.transData.transform((-.31, 0)))[0], 1, f"={model.rsquared_adj:.3f}", transform=ax.transAxes, fontsize=14, verticalalignment='top')

    # Adjust tick parameters for both axes
    ax.tick_params(axis='both', labelsize=12)  # Increase tick label font size

    # Remove top and right borders
    sns.despine()

      # Formula at the bottom
    ax.text(0.45, -0.2, formula_title, transform=ax.transAxes, fontsize=10, ha='center', va='top')

    # Adjust tick parameters for both axes
    ax.tick_params(axis='both', labelsize=12)

    # Remove top and right borders
    sns.despine()

    # Manually adjust the subplot parameters
    plt.subplots_adjust(left=0.15, right=0.95, top=0.85, bottom=0.25)  # Adjusted to make space for formula text


    if save is not None:
        save_path = f'/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/operation_regressions/figures/{save}.png'
        plt.savefig(save_path, dpi=300,  bbox_inches='tight')

    plt.show()


In [ ]:
reg_plots(sub_disp_ecc_acc_ev_means, reg_vars, ['brd_z_ave'], '#a8009d', 'Best Fitting Metrics Predicting Thought Problems', 
          'Thought Problems ~ dmn_mean + vn_mean_ecc + dmn_mean_ecc + clear_acc + fpcn_dmn_mean', save='top_formula')

In [ ]:
#!jupyter nbconvert --to script bootstrapped_regressions.ipynb 